## Comparison between the iterative Image Space Restoration Algorithm (ISRA) and the Richardson-Lucy Algorithm (RLA) 
Using the generated Voigt functions



In [ ]:
#required libraries
%matplotlib qt
import numpy as np
import hyperspy.api as hs
from ncempy.io import dm
import matplotlib.pyplot as plt
from scipy.signal import peak_widths, find_peaks
import math

Functions

In [ ]:
#convert text files
def txtconverter(numpy_array):
    file = str(numpy_array).replace('[','')
    file = file.replace(']','')
    data = np.fromstring(file, sep=',')
    return data

#sorting data into counts and eV
def find_counts(data):
    counts = data[1:-1:2]
    return counts

def find_ev(data):
    ev = data[0:-1:2]
    return ev

#plot the spectrum with HyperSpy
def hyperspy_plot(ev, counts):
    s = hs.signals.EELSSpectrum(counts)
    s.axes_manager[0].scale = np.diff(ev).mean()
    s.axes_manager[0].unit = 'eV'
    s.axes_manager[0].offset = ev[0]
    s.axes_manager[0].name = 'Energy'
    return s

#Normalize spectrum based on total electron counts
def normalizeS(S):
	S_norm = S/np.sum(S)
	return S_norm

#ISRA algorithm
def ISRA(iterations, PSF, Spec):
    ISRA5 = np.copy(Spec)
    ISRA1 = np.convolve(PSF, Spec, mode='same')
    for i in range(iterations):
        ISRA2 = np.convolve(PSF, ISRA5, mode='same')
        ISRA3 = np.convolve(PSF, ISRA2, mode='same')
        ISRA4 = np.divide(ISRA1, ISRA3)
        ISRA5 = np.multiply(ISRA4, ISRA5)
    return ISRA5

#Richardson-Lucy algorithm (code from Edson Bellido)
def RL(iterations, PSF, Spec):
    RL4 = np.copy(Spec)
    for i in range(iterations):
        RL1 = np.convolve(PSF, RL4, mode='same')
        RL2 = np.divide(Spec,RL1)
        RL3 = np.convolve(PSF, RL2, mode='same')
        RL4 = np.multiply(RL3, RL4)
    return RL4

#FWHM comparisons
def FWHM_testing(sigma, gamma, hs_signal, hs_deconvolved, height):
    
    peaks1, _ = find_peaks(hs_signal, height=1)
    results_half_signal = peak_widths(hs_signal, peaks1, rel_height=0.5)
    
    peaks2, _ = find_peaks(hs_deconvolved, height=height)
    results_half_deconvolved = peak_widths(hs_deconvolved, peaks2, rel_height=0.5)
    
    FWHM_signal = 4 / 1000 * results_half_signal[0]
    FWHM_deconvolved = 4 / 1000 * results_half_deconvolved[0]
    
    Lorentzian_FWHM = 2 * gamma
    Gaussian_FWHM = 2.335 * sigma
    
    relative_error = abs((FWHM_deconvolved[0] - Lorentzian_FWHM)/Lorentzian_FWHM*100)
    
    print("FWHM of signal =", FWHM_signal[0], "eV", 
          "\nFWHM of deconvolved =", FWHM_deconvolved[0], "eV", 
          "\nFWHM of Lorentzian =", Lorentzian_FWHM, "eV", 
          "\nRelative error =",  math.trunc(relative_error), "%\n")

Load files

In [ ]:
#load file as numpy array
Signal = np.loadtxt("D:\Downloads\Signal1.txt",dtype="str")
PSF = np.loadtxt("D:\Downloads\PSF1.txt", dtype='str')
Real = np.loadtxt("D:\Downloads\Real1.txt", dtype='str')

#convert text file to usable numpy array
signal = txtconverter(Signal)
psf = txtconverter(PSF)
real = txtconverter(Real)

#separate data into counts and ev
signal_counts = find_counts(signal)
psf_counts = find_counts(psf)
real_counts = find_counts(real)
ev = find_ev(signal)

Deconvolution

In [ ]:
ISRA_deconvolve = ISRA(10, psf_counts, signal_counts)
s_ISRA = hyperspy_plot(ev, ISRA_deconvolve)

RL_deconvolve = RL(10, psf_counts, signal_counts)
s_RL = hyperspy_plot(ev, RL_deconvolve)

Signal from before deconvolution and the real signal for comparison

In [ ]:
s_signal = hyperspy_plot(ev, signal_counts)
s_real = hyperspy_plot(ev, real_counts)

### FWHM comparisons <br> 
ISRA results with signal damping therefore a very small height is needed in order for the find_peaks function to work. This can be judged by looking at the plot. RL does not have this issue, usually a height of around 1 will work.

In [ ]:
print("RL")
FWHM_info_RL = FWHM_testing(0.1, 0.1, s_signal, s_RL, 1)
print("ISRA")
FWHM_info_ISRA = FWHM_testing(0.1, 0.1, s_signal, s_ISRA, 0.001)

Plot functions

In [ ]:
s_signal.plot()

In [ ]:
s_RL.plot()

In [ ]:
s_ISRA.plot()

In [ ]:
s_real.plot()